In [1]:
import sys
import os
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

# Avoids warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import tensorflow as tf

import ScalableLib.classifier.Multiband as multiband

2024-08-30 13:07:12.255829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-30 13:07:12.255863: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-30 13:07:12.256902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-30 13:07:13.199715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


the filtering of the times must be done in the spine, not in each band. Spit all the times then slect the best one. 

In [2]:
# To see if the system recognises the GPU
device = 1
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)

device_name = tf.config.experimental.get_device_details(devices[device])['device_name']
print("Using {}".format(device_name))

Using NVIDIA GeForce RTX 2080 Ti


Find the different folds and train a model using the stored data.

In [3]:
survey = 'Gaia'
path = os.path.join('../../02_CreateRecords/Gaia/Folds/*',)
folds = glob(path)
folds.sort()
folds

['../../02_CreateRecords/Gaia/Folds/Fold_1',
 '../../02_CreateRecords/Gaia/Folds/Fold_2',
 '../../02_CreateRecords/Gaia/Folds/Fold_3',
 '../../02_CreateRecords/Gaia/Folds/Fold_4',
 '../../02_CreateRecords/Gaia/Folds/Fold_5',
 '../../02_CreateRecords/Gaia/Folds/Fold_6',
 '../../02_CreateRecords/Gaia/Folds/Fold_7']

Create folder results

In [4]:
if not os.path.exists('./Results'):
    os.mkdir('./Results')


Define the arguments for all the models.

In [5]:
train_args = {
            'hidden_size_bands':[128, 128, 128],
            'hidden_size_central':[128, 128],
            'fc_layers_bands':[128,128,128],
            'fc_layers_central':[128,128,128], # Neurons of each layer
            'regression_size':[128, 128],#each element is a layer with that size.
            'buffer_size':10000,
            'epochs':1000,
            'num_threads':7,
            'batch_size':512,
            # 'batch_size':1024,
            'dropout':0.40,
            'lr':[[1e-3]*2, 0.5e-3], # [[band1, band2], central]
            'val_steps':50,
            'max_to_keep':0, # Not Used 
            'steps_wait':500, 
            'use_class_weights':False,# Not Used
            'mode' : 'classifier+regression',
            }
loss_weights = {'Class':300.0, 'T_eff':1.0,'Radius':1e0}
# loss_weights = {'Class':300.0, 'T_eff':20.0,'Radius':1e0}

callbacks_args = {'patience': 20,
                  'mode':'max',
                  'restore_best_weights':True,
                  'min_delta': 0.001
                 }
train_args_specific={
                    'phys_params': ['T_eff', 'Radius'],
                    'use_output_bands' : True,  # Working
                    'use_output_central' : False, # Not used
                    'use_common_layers' : False, # NOT Working
                    'bidirectional_central' : False,# Working
                    'bidirectional_band' : False,# Not Working
                    'layer_norm_params' : None, # Used to normalyze common layers
                    'use_gated_common' : False, # Working
                    'l1':0.0,
                    'l2':0.0,   
                    'N_skip' : 8, # Cannot be greater than the number of timesteps
                    'use_raw_input_central': True,
                    'train_steps_central' : 2,
                    'print_report' : True,
                    'loss_weights_central' : loss_weights,
                    'callbacks_args':callbacks_args
                    }



In [ ]:
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Set the save path for this fold. Create folder if needed
    path_results_fold = fold.replace('../02_CreateRecords', '.').replace('/Folds/', '/Results/')

    if not os.path.exists(path_results_fold):
        os.mkdir(path_results_fold)    

    train_args_specific['save_dir'] = path_results_fold
    train_args_specific['metadata_pre_path'] = os.path.join(fold, 'metadata_preprocess.json')
    train_args_specific['path_scalers'] =  os.path.join(fold,'scalers')
    # Define the train args
    train_args = {**train_args, **train_args_specific}

    train_files = os.path.join(fold, 'train/*.tfrecord')
    val_files = os.path.join(fold, 'val/*.tfrecord')
    test_files = os.path.join(fold, 'test/*.tfrecord')
    
    new = multiband.Network()    
    new.train(train_args, train_files, val_files, test_files)
    new.train_loop()


.././Gaia/Results/Fold_1/Models/20240830-1307
Start training


I0000 00:00:1725037652.118987   19146 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 2s 75ms/step - loss: 2757.5366 - Class_loss: 9.1335 - Pred_Radius_loss: 17.3930 - Pred_T_eff_loss: 0.1079 - Class_CentralAcc: 0.7401 - Class_CentralTop2: 0.8740 - Class_FinalAcc: 0.7638 - Class_FinalTop2: 0.8891 - Class_Final_FScore: 0.5948 - Pred_Radius_Masked_RMSE: 1.4659 - Pred_Radius_Masked_R2: 0.2667 - Pred_T_eff_Masked_RMSE: 0.2521 - Pred_T_eff_Masked_R2: 0.6407
